In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import joblib # To load the scaler

# --- Load the Trained Model and Scaler ---
MODEL_FILE = 'diabetes_model.keras'
SCALER_FILE = 'scaler.joblib'

def load_dependencies():
    """Loads the trained Keras model and the scaler from file."""
    if not os.path.exists(MODEL_FILE) or not os.path.exists(SCALER_FILE):
        print(f"Error: Model or scaler file not found.")
        print("Please run the 'train_model.py' script first to create the files.")
        return None, None

    print(f"Loading model from '{MODEL_FILE}'...")
    model = tf.keras.models.load_model(MODEL_FILE)

    print(f"Loading scaler from '{SCALER_FILE}'...")
    scaler = joblib.load(SCALER_FILE)

    print("Model and scaler loaded successfully.")
    return model, scaler

def interactive_prediction(model, scaler):
    """An interactive function to predict diabetes risk based on user input."""
    if model is None or scaler is None:
        return

    print("\n--- Interactive Diabetes Risk Prediction ---")
    print("Please enter the following details:")

    try:
        # --- Collect and Prepare User Input ---
        age = float(input("Enter Age (e.g., 35): "))
        gender_str = input("Enter Gender ('female' or 'male'): ").lower()
        gender = 0 if gender_str == 'female' else 1
        history_str = input("Family history of diabetes? ('yes' or 'no'): ").lower()
        family_history = 1 if history_str == 'yes' else 0
        bp_str = input("Have you ever had high blood pressure? ('yes' or 'no'): ").lower()
        high_bp = 1 if bp_str == 'yes' else 0
        exercise_str = input("Enter exercise frequency ('never', '1-2 times', '3-4 times', '5 or more times'): ").lower()
        exercise_map = {'never': 0, '1-2 times': 1, '3-4 times': 2, '5 or more times': 3}
        exercise_freq = exercise_map.get(exercise_str, 0)
        weight_kg = float(input("Enter Weight in Kilograms (e.g., 70): "))
        height_cm = float(input("Enter Height in Centimeters (e.g., 175): "))

        # --- Preprocess the Input Data ---
        height_m = height_cm / 100
        if height_m == 0:
            print("Error: Height cannot be zero.")
            return
        bmi = weight_kg / (height_m ** 2)

        input_data = pd.DataFrame({
            'Age': [age], 'Gender': [gender], 'Family_History_Diabetes': [family_history],
            'High_Blood_Pressure': [high_bp], 'Exercise_Frequency': [exercise_freq], 'BMI': [bmi]
        })

        # IMPORTANT: Scale the input data using the loaded scaler
        columns_to_scale = ['Age', 'BMI']
        input_data[columns_to_scale] = scaler.transform(input_data[columns_to_scale])

        # --- Make Prediction ---
        prediction_prob = model.predict(input_data)[0][0]
        prediction_class = "High Risk" if prediction_prob > 0.5 else "Low Risk"

        # --- Display the Result ---
        print("\n--- Prediction Result ---")
        print(f"The model predicts a '{prediction_class}' of diabetes.")
        print(f"Confidence Score (Probability): {prediction_prob:.2%}")
        print("-------------------------\n")

    except ValueError:
        print("\nError: Invalid input. Please enter numbers for age, weight, and height.\n")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}\n")

# --- Main Loop ---
if __name__ == '__main__':
    loaded_model, loaded_scaler = load_dependencies()
    if loaded_model:
        while True:
            interactive_prediction(loaded_model, loaded_scaler)
            another = input("Do you want to make another prediction? (yes/no): ").lower()
            if another != 'yes':
                print("Exiting interactive session.")
                break